In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
try:
    global jsondata
    with open('data.json') as f:
        jsondata = json.load(f)
except:
    print("file not found!!")
    

In [4]:
def wrangle_data(rawdata):
    df=pd.DataFrame.from_dict(jsondata['data'])
    df.dropna(inplace=True)
    df['date']= pd.to_datetime(df['date'])
    df.index=df['date']
    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    filters = [
    'areaType=overview'
    ]
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "cumDeaths28DaysByDeathDateRate"    
    }
    api = Cov19API(filters=filters, structure=structure)
    read_api_data=api.get_json()
    return read_api_data

# Refreshing Api Data

if data is not avalible or become old please click on refresh button! 

In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    try:
        apidata=access_api()
        # wrangle the data and overwrite the dataframe for plotting
        global df
        df=wrangle_data(apidata)
        # the graph won't refresh until the user interacts with the widget.
        # this function simulates the interaction, see Graph and Analysis below.
        # you can omit this step in the first instance
        refresh_graph()
        # after all is done, you can switch the icon on the button to a "check" sign
        # and optionally disable the button - it won't be needed again. You can use icons
        # "unlink" or "times" and change the button text to "Unavailable" in case the 
        # api call fails.
        apibutton.icon="check"
        apibutton.disabled=True
    except Exception as e:
        print(e)
        apibutton.description='Unavailable'
        apibutton.button_style='danger'
        apibutton.icon='fa-chain-broken'
        
    
apibutton=wdg.Button(
    description='Refresh', # you may want to change this...
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='fa-download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='success', description='Refresh', icon='download', style=ButtonStyle(), tooltip='Keep calm…

# Visualizing graphs

select <b>cases/deaths</b> from first drop down<br>
select data by <b>month</b> from second drop down<br>
In the second drop down all means data of all months(year)

In [8]:
def plot_random_walk(walk):
    df[walk].plot()
    
whichwalk=wdg.Dropdown(
    options=['cases', 'deaths'],
    value='cases',
    description='cases/deaths ',
    disabled=False,
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=whichwalk.value
    if current==whichwalk.options[0]:
        other=whichwalk.options[1]
    else:
        other=whichwalk.options[0]
    whichwalk.value=other # forces the redraw
    whichwalk.value=current # now we can change it back

    
# graph=wdg.interactive_output(plot_random_walk, {'walk': whichwalk})
    
# display(whichwalk, graph)

# graph with selected date

def plot_random_walk_date(walk,date):
    if date=='all':
        df[walk].plot()
    else:
        date_df=df['2020-'+update_date()]
        date_df[walk].plot()

def update_date():
    return date_dropDowm.value
# date drop down
option_dates=[]

option_dates=['all']+df.index.month.unique().astype('str').tolist()
# option_dates.append('all')
date_dropDowm=wdg.Dropdown(
    options=option_dates,
    value=option_dates[0],
    description='Month',
    disabled=False,
)

graph=wdg.interactive_output(plot_random_walk_date, {'walk': whichwalk,'date':date_dropDowm})
    
display(whichwalk,date_dropDowm, graph)

Dropdown(description='cases/deaths ', options=('cases', 'deaths'), value='cases')

Dropdown(description='Month', options=('all', '11', '10', '9', '8', '7', '6', '5', '4', '3'), value='all')

Output()

**Author and Copyright Notice** <br> 
credit: *Feima Ndoeka* <br>
data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*